Personalization Flow
- Take different behavior -> view -> purchase (time?)
- Aggregate and create weight
- Display -> food bubble and playlist

In [52]:
import numpy as np
import pandas as pd
import pickle
import json

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from config import get_image_url

In [50]:
list_embd = np.load('./data/list_embd.npy')
list_keys = np.load('./data/list_keys.npy')
len(list_keys)

756

In [13]:
search_algo = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(list_embd)

In [57]:
shop_encoder = {}
for i, key in enumerate(list_keys):
    shop_encoder[key] = list_embd[i]

In [62]:
with open('./data/shop_encoder.pkl', 'wb') as enc:
    pickle.dump(shop_encoder, enc,)

In [14]:
with open('./model/nn_model.pkl', 'wb') as m:
    pickle.dump(search_algo, m)

In [107]:
class recommender():

    def __init__(self, df):
        self.model = pickle.load(open('./model/'+ 'nn_model' + '.pkl', 'rb')) # Sklearn model
        self.encoder = pickle.load(open('./model/'+ 'shop_encoder' + '.pkl', 'rb')) # Dict
        self.df = df[df['favorite'] == True]

    def preprocess(self):
        encoded_shop_ids = [self.encoder[v] for v in self.df.shop_id.values]

        return np.mean(encoded_shop_ids, axis=0) 
    
    def postprocess(self, embedding_position):
        # Get key or shop_id from position
        keys = list(self.encoder.keys())
    
        list_of_shopIds = [keys[i] for i in embedding_position]
        return list_of_shopIds

    def get_playlist(self, k):

        embdedding_vector = self.preprocess()
        k = 20 if k > 20 else k

        topK = self.model.kneighbors(X=[embdedding_vector], n_neighbors=20, return_distance=False)

        topShops = self.postprocess(topK[0])
        
        return topShops

In [114]:
df_shop.loc[10:, 'favorite'] = False
df_shop.loc[:40, 'favorite'] = True

/var/folders/1d/ych47sfj3755tjsj8j85ngf40000gn/T/ipykernel_58322/1906199920.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_shop.loc[10:, 'favorite'] = False


In [109]:
rec = recommender(df_shop)

In [110]:
rec.get_playlist(10)

[42154,
 123495,
 176680,
 98344,
 41629,
 62448,
 36083,
 17285,
 23240,
 67255,
 24142,
 117673,
 80483,
 220358,
 211594,
 101087,
 239977,
 36971,
 106861,
 149362]

In [136]:
df_shop = pd.read_csv("./data/shop_meta_data.csv")

In [139]:
df_shop.to_csv("./data/shop_meta_data.csv", index=False)

In [137]:
topHit_emoji = {
    'cooked_to_order': '🍳',
    'noodles': '🍜',
    'drinks': '🍹',
    'snacks': '🍿',
    'bakery': '🥐',
    'grill': '🍖',
    'coffee/tea': '☕',
    'single_dish': '🍛',
    'ramen': '🍜',
    'congee/rice_soup': '🥣',
    'dessert': '🍰',
    'fried_chicken': '🍗',
    'shaved_ice': '🍧',
    'sushi': '🍣',
    'milk_tea': '🧋',
    'fruits': '🍎',
    'somtum': '🥗',
    'steak': '🥩',
    'shabu/bbq': '🍲',
    'dimsum': '🥟',
    'fast_food': '🍔',
    'burgers': '🍔',
    'spaghetti': '🍝',
    'side_dish': '🧆',
    'ice-cream': '🍦',
    'seafood': '🦞',
    'healthy': '🥗',
    'pizza': '🍕',
    'pickled_food': '🥒',
    'ice': '🧊'
}

In [138]:
df_shop = df_shop[df_shop.shop_category.isin(list(topHit_emoji.keys()))]

In [116]:
df = df_shop[df_shop['favorite']==True]

In [5]:
df_shop = pd.read_csv("./data/shop_meta_data.csv", usecols=['shop_photo', 'shop_id', 'merchant_name', ])

In [9]:
df_shop['shop_photo'].apply(get_image_url)[0]

'https://static.robinhood.in.th/merchant/shop/1616228248540_48R8J8BLOS.jpg'

In [133]:
purchased_list = list(df[['shop_name', 'shop_category']].groupby('shop_category').count().index)

In [134]:
for i in range(len(topHit)):
    if (topHit[i] not in purchased_list) & (len(purchased_list) <= 9):
        purchased_list.append(topHit[i])

In [135]:
len(purchased_list)

10